In [13]:
import numpy as np
import h5py 
import os
from tensorflow import keras
import pdb
import glob
import pandas as pd
from geospacepy import special_datetime


# Import the Conjunction Data

In [3]:
hd_dir = os.path.join('/home/matsuo/amgeo_dev/LBH_to_eflux/LBH_to_eflux/','conjunction_data')
file_dir = os.path.join(hd_dir,'*.hdf5')
conjunc_files = glob.glob(file_dir)
conjunc_files = np.sort(conjunc_files) #sort the files by time 

ele_diff_energy_flux_arr,ion_diff_energy_flux_arr = np.empty((0,19)) ,np.empty((0,19))
ele_flux, ion_flux = [],[]
ele_mean, ion_mean = [],[]
ssusi_lbhl, ssusi_lbhs, ssusi_lyman = [],[], []
jds, lons, lats = [], [], []
sat_nums, passes, hemis = [], [], []

for file_name in conjunc_files:
    with h5py.File(file_name, 'r') as f:
        jds.extend(f['jds'][:])
        passes.extend(f['pass_num'][:])
        sat_nums.extend(f['sat_no'][:])
        lons.extend(f['lons'][:])
        lats.extend(f['lats'][:])
#         hemis.extend(f['hemi'][:])
        #input data
        ssusi_lbhl.extend(f['LBHL_interped'][:])
        ssusi_lbhs.extend(f['LBHS_interped'][:])
        ssusi_lyman.extend(f['LYMAN_interped'][:])
        #output 

        ele_flux.extend(f['ele_total_energy_flux'][:])
        ion_flux.extend(f['ion_total_energy_flux'][:])
        
        ele_mean.extend(f['ele_mean_energy'][:])
        ion_mean.extend(f['ion_mean_energy'][:])


In [5]:
d = {'jds': jds, 'passes': passes, 'sat_nums' : sat_nums, 'lons' : lons, 'lats' : lats, 
     'lbhl' : ssusi_lbhl,'lbhs' : ssusi_lbhs, 'lyman' : ssusi_lyman,
    'ion_total_flux' : ion_flux, 'ele_total_flux' : ele_flux}
df = pd.DataFrame(data=d)

In [18]:
enddt = special_datetime.jd2datetime(np.nanmax(df['jds']))
startdt = special_datetime.jd2datetime(np.nanmin(df['jds']))

In [15]:
unique_passes = np.unique(df['passes'])

# Feature Engineering

## Geomagnetic Indices Features

### Get IMF components and useful activity indices

For this I use a useful tool by Liam Kilcommons that allows you to autodownload NASA omniweb data.

In [8]:
from nasaomnireader import omnireader 

Traceback (most recent call last):
  File "/home/matsuo/amgeo_dev/AMGeO/src/nasaomnireader/nasaomnireader/__init__.py", line 5, in <module>
    from nasaomnireader.omnireader_config import config
ModuleNotFoundError: No module named 'nasaomnireader.omnireader_config'

Solar wind data files will be saved to /home/matsuo/.local/share/nasaomnireader


In [27]:
def download_omni_data(startdt,enddt):
    omni_data = {}
    freq = '5min'
    indices =['BY_GSM','BZ_GSM','AE_INDEX','AL_INDEX']
    omniInt = omnireader.omni_interval(startdt,enddt,freq)
    jd_arr = omniInt['Epoch']
    for index in indices:
        omni_data[index] = omniInt[index] 
    return jd_arr,omni_data

jd_arr, omni_data = download_omni_data(startdt,enddt)

Created interval between 2014-02-17 and 2014-02-23, cadence 5min, start index 4610, end index 6624


### Interpolate observations 

Now I need to interpolate these index values to the observation times

## Smoothing

Applying smoothing to the time series helps reduce noise

Get rid of nan rows 

## Scaling and Normalizing

## Model Structure

In [ ]:
def get_first_electron_flux_model(shape(2), name = 'FUV'):
    
    
    return

def get_model(n_inputs,n_outputs):
    
    inputs = keras.Input(shape=(3),name ='FUV')

#     lbh_input = keras.Input(shape=(2), name = 'LBH')(inputs[0,1])
#     lbh_and_lyman_input = keras.Input(shape=(1), name = 'LBH_and_Lyman')
    je1_first_layer = keras.layers.Dense(8, kernel_initializer='normal',activation = 'relu')(inputs)
    je1_hidden_layer = keras.layers.LeakyReLU(32)(je1_first_layer)
    je1_output = keras.layers.Dense(1)(je1_hidden_layer)
    
    
    ji1_first_layer = keras.layers.Dense(8, kernel_initializer='normal',activation = 'relu')(inputs)
    ji1_hidden_layer = keras.layers.LeakyReLU(32)(ji1_first_layer)
    ji1_output = keras.layers.Dense(1)(ji1_hidden_layer)
    
    je2_input = keras.layers.Concatenate(axis=-1)([inputs,je1_output,ji1_output])
    je2_first_layer = keras.layers.Dense(8, kernel_initializer='normal',activation = 'relu')(je2_input)
    je2_hidden_layer = keras.layers.LeakyReLU(32)(je2_first_layer)
    je2_output = keras.layers.Dense(1)(je2_hidden_layer)

    ji2_input = keras.layers.Concatenate(axis=-1)([inputs,je1_output,ji1_output])
    ji2_first_layer = keras.layers.Dense(8, kernel_initializer='normal',activation = 'relu')(ji2_input)
    ji2_hidden_layer = keras.layers.LeakyReLU(32)(ji2_first_layer)
    ji2_output = keras.layers.Dense(1)(ji2_hidden_layer)
    
    outputs = keras.layers.Concatenate(axis=-1)([je2_output,ji2_output])
    model = keras.Model(inputs = inputs, outputs = outputs)

    optimizer = keras.optimizers.Adam(learning_rate = 0.01)
    loss = [keras.losses.MeanSquaredError()]
    # Compile the network :
    model.compile(loss='mse', optimizer= optimizer, metrics=['mse','mean_absolute_percentage_error'])

    return model

In [ ]:
model = get_model(3,2)

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, "my_first_model.png")


Model Training with K folds Cross Val